# ST IT Cloud - Data and Analytics Test LV.4

Esse teste deve avaliar alguns conceitos de big data e a qualidade técnica na manipulacão de dados, otimização de performance, trabalho com arquivos grandes e tratamento de qualidade.

## Passo a passo

-Disponibilizamos aqui 2 cases para serem desenvolvidos, leia os enunciados dos problemas, desenvolver os programas, utilizando a **stack definida durante o processo seletivo**, para entregar os dados de acordo com os requisitos descritos abaixo.

**Faz parte dos critérios de avaliacão a pontualidade da entrega. Implemente até onde for possível dentro do prazo acordado.**

**Os dados de pessoas foram gerados de forma aleatória, utilizando a biblioteca FakerJS, FakerJS-BR e Faker**

LEMBRE-SE: A entrega deve conter TODOS os passos para o avaliador executar o programa (keep it simple).


# TESTE PRÁTICO

**Problema 1**: Você está recebendo o arquivo 'dados_cadastrais_fake.csv' que contem dados cadastrais de clientes, mas para que análises ou relatórios sejam feitos é necessário limpar e normalizar os dados. Além disso, existe uma coluna com o número de cpf e outra com cnpj, você precisará padronizar deixando apenas dígitos em formato string (sem caracteres especiais), implementar uma forma de verificar se tais documentos são válidos sendo que a informação deve se adicionada ao dataframe em outras duas novas colunas.

Após a normalização, gere reports que respondam as seguintes perguntas:
- Quantos clientes temos nessa base?
- Qual a média de idade dos clientes?
- Quantos clientes nessa base pertencem a cada estado?
- Quantos CPFs válidos e inválidos foram encontrados?
- Quantos CNPJs válidos e inválidos foram encontrados?

Ao final gere um arquivo no formato csv e um outro arquivo no formato parquet chamado (problema1_normalizado), eles serão destinados para pessoas distintas.


#### Módulos Python

In [246]:
%pip install unidecode
%pip install fastparquet

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [247]:
from unidecode  import unidecode
import pandas as pd
import csv
import re

#### CSV Path

In [248]:
dados_csv = "dados_cadastrais_fake.csv"

#### VARIÁVEIS GLOBAIS

In [249]:
LEN_CPF = [10,11]
LEN_CNPJ = 14

In [250]:
REGEX_CPF = r'([0-9]{2}.[0-9]{3}.[0-9]{3}-[0-9]{2})|([0-9]{3}.[0-9]{3}.[0-9]{3}-[0-9]{2})'
REGEX_CNPJ = r'([0-9]{2}.[0-9]{3}.[0-9]{3}/[0-9]{4}-[0-9]{2})'
REGEX_FINAL = r'[^0-9]'

#### DICT DE UF

In [251]:
DICT_UF = {
    'RO' : 'RONDONIA',
    'AC' : 'ACRE',
    'AM' : 'AMAZONAS',
    'RR' : 'RORAIMA',
    'PA' : 'PARA',
    'AP' : 'AMAPA',
    'TO' : 'TOCANTIS',
    'MA' : 'MARANHAO',
    'PI' : 'PIAUI',
    'CE' : 'CEARA',
    'RN' : 'RIO GRANDE DO NORTE',
    'PB' : 'PARAIBA',
    'PE' : 'PERNAMBUCO',
    'AL' : 'ALAGOAS',
    'SE' : 'SERGIPE',
    'BA' : 'BAHIA',
    'MG' : 'MINAS GERAIS',
    'ES' : 'ESPIRITO SANTO',
    'RJ' : 'RIO DE JANEIRO',
    'SP' : 'SAO PAULO',
    'PR' : 'PARANA',
    'SC' : 'SANTA CATARINA',
    'RS' : 'RIO GRANDE DO SUL',
    'MS' : 'MATO GROSSO DO SUL',
    'MT' : 'MATO GROSSO',
    'GO' : 'GOIAS',
    'DF' : 'DISTRITO FEDERAL'
}

#### FUNÇÕES DE PROCESSAMENTO

In [252]:
def set_csvfile_to_dataframe(file_path):
    df = pd.read_csv(file_path, 
                     delimiter=';', 
                     quoting=csv.QUOTE_NONE,  
                     warn_bad_lines=True)
    
    return df

In [253]:
def set_csvfile_to_parquet(df_in):
    df_in.to_parquet('problema1_normalizado.parquet.gzip',compression='gzip')

In [254]:
def set_regex_to_clean_str(data_in):
    if type(data_in) == str:
        data_str = data_in.upper()
    else:
        data_str = str(data_in.upper())
    return re.sub(REGEX_FINAL, "", data_str)

In [255]:
def set_unidecode_to_str(data_in):
    if type(data_in) == str:
        data_str = data_in.upper()
    else:
        data_str = str(data_in.upper())
    return unidecode(data_str)

In [256]:
def is_cpf_valid(data_in):
    if type(data_in) == str:
        data_str = data_in.upper()
    else:
        data_str = str(data_in.upper())
    
    if re.search(REGEX_CPF, data_str):
        data_str_clean = set_regex_to_clean_str(data_str)
        if len(data_str_clean) in LEN_CPF:
            return True
        else:
            return False
    else:
        return False

In [257]:
def is_cnpj_valid(data_in):
    if type(data_in) == str:
        data_str = data_in.upper()
    else:
        data_str = str(data_in.upper())
    
    if re.search(REGEX_CNPJ, data_str):
        data_str_clean = set_regex_to_clean_str(data_str)
        if len(data_str_clean) == LEN_CNPJ:
            return True
        else:
            return False
    else:
        return False

In [258]:
def set_uf_adjust (data_in):
    uf_list = list(DICT_UF.keys())
    if type(data_in) == str:
        data_str = data_in.upper()
    else:
        data_str = str(data_in.upper())
    
    if data_str not in uf_list:
        for uf in uf_list:
            if (DICT_UF[uf] == data_str)\
               |(DICT_UF[uf] in data_str)\
                |(data_str in DICT_UF[uf]):
        
                return uf
        return None
    else:
        return data_str

In [259]:
def set_dataset_normalization (data_in):
    data_in['nomes']  = data_in['nomes'].astype(str)
    data_in['cidade'] = data_in['cidade'].astype(str)
    data_in['estado'] = data_in['estado'].astype(str)
    data_in['cpf']    = data_in['cpf'].astype(str)
    data_in['cnpj']   = data_in['cnpj'].astype(str)

    data_in['nomes']  = data_in['nomes'].apply(set_unidecode_to_str)
    data_in['cidade'] = data_in['cidade'].apply(set_unidecode_to_str)
    data_in['estado'] = data_in['estado'].apply(set_unidecode_to_str)

    data_in['estado'] = data_in['estado'].apply(set_uf_adjust)

    data_in['nomes']  = data_in['nomes'].str.upper()
    data_in['cidade'] = data_in['cidade'].str.upper()
    data_in['estado'] = data_in['estado'].str.upper()

    data_in['cpf_valido']  = data_in['cpf'].apply(is_cpf_valid)
    data_in['cnpj_valido'] = data_in['cnpj'].apply(is_cnpj_valid)

    data_in['cpf']  = data_in['cpf'].apply(set_regex_to_clean_str)
    data_in['cnpj'] = data_in['cnpj'].apply(set_regex_to_clean_str)

    data_in = data_in.drop_duplicates()
    return data_in

In [260]:
def get_age_mean_col (df_in):
    return df_in['idade'].mean(axis = 0, skipna = False)

In [261]:
def get_count_customers (df_in):
    df_cpf = df_in['cpf'].drop_duplicates()
    return len(df_cpf)

In [262]:
def get_count_estado (df_in):
    df_est = df_in.groupby(['estado']).size()
    return df_est

In [263]:
def get_count_cpf_valido (df_in):
    df_cpf = df_in[df_in['cpf_valido'] == True]
    count = len(df_cpf)
    return count

In [264]:
def get_count_cpf_invalido (df_in):
    df_cpf = df_in[df_in['cpf_valido'] == False]
    count = len(df_cpf)
    return count

In [265]:
def get_count_cnpj_valido (df_in):
    df_cnpj = df_in[df_in['cnpj_valido'] == True]
    count = len(df_cnpj)
    return count

In [266]:
def get_count_cnpj_invalido (df_in):
    df_cnpj = df_in[df_in['cnpj_valido'] == False]
    count = len(df_cnpj)
    return count

#### GERAÇÃO DE DF NORMALIZADO

In [267]:
df_csv = set_csvfile_to_dataframe(dados_csv)
df_csv_norm = set_dataset_normalization(df_csv)
df_csv_norm

C:\Users\engda\AppData\Local\Temp\ipykernel_103080\3380420549.py:2: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(file_path,


,nomes,idade,cidade,estado,cpf,cnpj,cpf_valido,cnpj_valido
0,DENNIS DANIELS,31,ACRELANDIA,AC,97566536800,06589184909526,False,False
1,LEAH BECKER,42,AGUA BRANCA,AL,42526380707,25673336235020,True,True
2,SALLY FORD,18,ALVARAES,AM,34647754103,26543101702989,False,False
3,COLLEEN DUNCAN,21,SERRA DO NAVIO,AP,25253156003,19062080510098,True,True
4,JEFF STEPHENSON,73,ABAIRA,BA,49668886542,97794530015384,False,False
...,...,...,...,...,...,...,...,...
9995,REBEKAH MITCHELL PHD,55,ABAIARA,CE,74482262234,16740076932975,True,True
9996,LISA PARRISH JR.,73,BRASILIA,DF,10683395190,32246978843482,False,False
9997,MICHAEL YOUNG MD,87,AFONSO CLAUDIO,ES,53822363804,86601303758088,True,True
9998,KEVIN WATSON DDS,82,ABADIA DE GOIAS,GO,11632512408,08651414023648,False,False


#### GERAÇÃO DE PARQUET

In [268]:
set_csvfile_to_parquet(df_csv_norm)

#### MÉTRICAS

##### 1) MÉDIA DE IDADE

In [269]:
age_mean = get_age_mean_col(df_csv_norm)
age_mean

53.7831

##### 2) QUANTIDADE DE CLIENTES

In [270]:
count_cust = get_count_customers (df_csv_norm)
count_cust

10000

##### 3) QUANTIDADE DE REGISTROS POR ESTADO

In [271]:
df_count_estado = get_count_estado (df_csv_norm)
df_count_estado

estado
AC    371
AL    371
AM    371
AP    371
BA    371
CE    371
DF    371
ES    371
GO    371
MA    371
MG    370
MS    370
MT    370
PA    370
PB    370
PE    370
PI    370
PR    370
RJ    355
RN    370
RO    370
RR    370
RS    370
SC    370
SE    370
SP    364
TO    370
dtype: int64

##### 4) QUANTIDADE DE CPF VÁLIDOS E INVÁLIDOS

In [272]:
count_cpf_valido = get_count_cpf_valido(df_csv_norm)
count_cpf_valido

5000

In [273]:
count_cpf_invalido = get_count_cpf_invalido(df_csv_norm)
count_cpf_invalido

5000

##### 5) QUANTIDADE DE CNPJ VÁLIDOS E INVÁLIDOS

In [274]:
count_cnpj_valido = get_count_cnpj_valido(df_csv_norm)
count_cnpj_valido

5000

In [275]:
count_cnpj_invalido = get_count_cnpj_invalido(df_csv_norm)
count_cnpj_invalido

5000

**Problema 2**: Você deverá implementar um programa, para ler, tratar e particionar os dados.

O arquivo fonte está disponível em `https://st-it-cloud-public.s3.amazonaws.com/people-v2_1E6.csv.gz`

### Data Quality

- Higienizar e homogenizar o formato da coluna `document`
- Detectar através da coluna `document` se o registro é de uma Pessoa Física ou Pessoa Jurídica, adicionando uma coluna com essa informação
- Higienizar e homogenizar o formato da coluna `birthDate`
- Existem duas colunas nesse dataset que em alguns registros estão trocadas. Quais são essas colunas? 
- Corrigir os dados com as colunas trocadas
- Além desses pontos, existem outras tratamentos para homogenizar esse dataset. Aplique todos que conseguir.

### Agregação dos dados

- Quais são as 5 PF que mais gastaram (`totalSpent`)? 
- Qual é o valor de gasto médio por estado (`state`)?
- Qual é o valor de gasto médio por `jobArea`?
- Qual é a PF que gastou menos (`totalSpent`)?
- Quantos nomes e documentos repetidos existem nesse dataset?
- Quantas linhas existem nesse dataset?

### Particionamento de dados tratados com as regras descritas em `DATA QUALITY`

- Particionar em arquivos PARQUET por estado (`state`)
- Particionar em arquivos CSV por ano/mes/dia de nascimento (`birthDate`)